In [1]:
cd /Data1/hmd2/notebooks_th/GRU_loss_add

/Data1/hmd2/notebooks_th/GRU_loss_add


In [2]:
ls

__pycache__/       main_BCE_loss.py       main_two_loss.py     models/
dataset.py         main_loss.py           main_two_loss_22.py  test.py
evaluate_model.py  main_no_frame_loss.py  metric.py            utils.py
main.py            main_slice_add.py      model.py             전처리.ipynb


In [3]:
from dataset import *
import pathlib

## 기존 전처리

In [4]:
data_folder = pathlib.Path("/Data2/murmur/train/")
num_k = 4
val_fold_num= 0

In [5]:
patient_df = load_patient_files(data_folder, stop_frac= 1)

patient_df = create_folds(patient_df, num_k)
recording_df = patient_df_to_recording_df(patient_df)

recording_df_gq = recording_df[recording_df["patient_murmur_label"] != "Unknown"] # 'Holosystolic', 'Early-systolic', 'Mid-systolic'
recording_df_bq = recording_df[recording_df["patient_murmur_label"] == "Unknown"] # murmur_timing : nan
    

train_recording_df = recording_df_gq[recording_df_gq['val_fold'] != val_fold_num]
val_recording_df = recording_df_gq[recording_df_gq["val_fold"] == val_fold_num]

In [6]:
train_recording_df.head()

,murmur_timing,rec_murmur_label,patient_murmur_label,outcome_label,val_fold
recording,,,,,
2530_AV,nan,Absent,Absent,Abnormal,1
2530_PV,nan,Absent,Absent,Abnormal,1
2530_TV,nan,Absent,Absent,Abnormal,1
2530_MV,nan,Absent,Absent,Abnormal,1
9979_AV,Holosystolic,Present,Present,Abnormal,2


In [7]:
train_recording_df.patient_murmur_label.unique()

array(['Absent', 'Present'], dtype=object)

In [8]:
shared_feature_cache = {}

train_dataset = CustomDataset(
                data_folder= data_folder, 
                recording_paths= train_recording_df.index, 
                murmur_labels= train_recording_df.patient_murmur_label, 
                outcome_labels= train_recording_df.outcome_label,
                timings= train_recording_df.murmur_timing,
        sequence_length= int(TRAINING_SEQUENCE_LENGTH / WINDOW_STEP), # Default: int(TRAINING_SEQUENCE_LENGTH / WINDOW_STEP)
        cache=shared_feature_cache
    )


val_dataset = CustomDataset(
                data_folder= data_folder, 
                recording_paths= val_recording_df.index, 
                murmur_labels= val_recording_df.patient_murmur_label, 
                outcome_labels= val_recording_df.outcome_label,
                timings= val_recording_df.murmur_timing,
        sequence_length= int(TRAINING_SEQUENCE_LENGTH / WINDOW_STEP), # Default: int(TRAINING_SEQUENCE_LENGTH / WINDOW_STEP)
        cache=shared_feature_cache
    )


In [9]:
# ex)39403_AV
len(val_recording_df.index), len(val_dataset)

(590, 590)

In [10]:
feature , frame_label, murmur_label, outcome_label = next(iter(train_dataset))

/opt/conda/lib/python3.10/site-packages/torch/functional.py:641: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343967769/work/aten/src/ATen/native/SpectralOps.cpp:862.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


In [11]:
print(f"Spectogram.shape is {feature.shape}\nframe_label.shape is {frame_label.shape}")
print(f"murmur_label is {murmur_label}\noutcome_label is {outcome_label}")

Spectogram.shape is torch.Size([40, 300])
frame_label.shape is torch.Size([300])
murmur_label is 0
outcome_label is 1


In [12]:
def batch_whole_dataset(dataset: CustomDataset):
    num_examples = len(dataset)
    return collate_fn_multi_output([dataset[i] for i in range(num_examples)])

def collate_fn_multi_output(x):
    max_length = 0
    all_features = []
    all_frame_labels = []
    all_murmur_labels = []
    all_outcome_labels = []
    #actual_lengths = []

    for feature, frame_label, murmur_label, outcome_label in x:
        all_features.append(feature)
        all_frame_labels.append(frame_label)
        all_murmur_labels.append(murmur_label)
        all_outcome_labels.append(outcome_label)
        max_length = max(max_length, len(frame_label))
        #actual_lengths.append(len(frame_label))
    
    padded_features = all_features[0].new_full((len(x), all_features[0].shape[0], max_length), 0.0)
    
    for i, feature in enumerate(all_features): 
        padded_features[i, ..., : feature.shape[-1]] = feature

        padded_labels = all_frame_labels[0].new_full((len(x), max_length), -1)
    
    for i, frame_label in enumerate(all_frame_labels):
        padded_labels[i, : frame_label.shape[-1]] = frame_label

    return padded_features, padded_labels, all_murmur_labels, all_outcome_labels  # torch.as_tensor(actual_lengths)

# feature , frame_label, murmur_label, outcome_label 

In [13]:
padded_features, padded_labels, all_murmur_labels, all_outcome_labels = batch_whole_dataset(train_dataset)

In [14]:
# 전부 다 [40, 300] shape이 맞는지??
for i in padded_features:
    if i.shape[0] != 40 or i.shape[1] != 300:
        print(i.shape)

In [15]:
# 전부 다 길이가 300이 맞는지??
for i in padded_labels:
    if i.shape[0] != 300:
        print(i.shape)

In [16]:
# murmur, outcome label 동일!
len(all_murmur_labels), len(all_outcome_labels)

(1813, 1813)

In [17]:
#class_weights = calculate_class_weights_wo(train_dataset)

In [18]:
#class_weights

#### 클래스별 가중치도 똑같이 나온다!

In [19]:
train_dataset_origin = RecordingDataset(data_folder,
                                 train_recording_df.index,
                                 train_recording_df.patient_murmur_label, 
                                 train_recording_df.murmur_timing,
                                 sequence_length= int(TRAINING_SEQUENCE_LENGTH / WINDOW_STEP),
                                 cache=shared_feature_cache)
class_weights = calculate_class_weights(train_dataset_origin)
class_weights

tensor([4.6811, 4.4460, 5.7195, 2.5866, 0.0000])

#### 로더에서도 문제없음

In [44]:
train_dataloader = torch.utils.data.DataLoader(
        train_dataset, batch_size= 32, shuffle=True, 
    )

val_dataloader = torch.utils.data.DataLoader(
        val_dataset, batch_size=15 , shuffle=False,
    )
    

In [46]:
len(train_dataset)

1813

In [21]:
len(train_dataset), len(val_dataset)

(1813, 590)

In [42]:
len(train_dataloader)

57

In [22]:
stack = 0

while True:
    feature, frame_label, murmur_label, outcome_label = next(iter(train_dataloader))
    stack += 1
    
    if stack == 1813:
        break

stack

1813

In [23]:
stack = 0

while True:
    feature, frame_label, murmur_label, outcome_label = next(iter(val_dataloader))
    stack += 1
    
    if stack == 590:
        break

stack

590

## 모델

In [24]:
class GRU_multi_output(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self.rnn = nn.GRU(
            input_size=calc_frequency_bins(),
            hidden_size=60,
            num_layers=3,
            batch_first=True,
            bidirectional=True,
            dropout=0.1
        )
        self.frame_linear = nn.Sequential(
            nn.Linear(120, 60),
            nn.Tanh(),
            nn.Dropout(0.1),
            nn.Linear(60, 40),
            nn.Tanh(),
            nn.Dropout(0.1),
            nn.Linear(40, 5)
        )
        self.murmur_linear = nn.Sequential(
            nn.Linear(60, 40),
            nn.Tanh(),
            nn.Dropout(0.1),
            nn.Linear(40, 10),
            nn.Tanh(),
            nn.Dropout(0.1),
            nn.Linear(10, 2)
            )
        self.outcome_linear = nn.Sequential(
            nn.Linear(60, 40),
            nn.Tanh(),
            nn.Dropout(0.1),
            nn.Linear(40, 10),
            nn.Tanh(),
            nn.Dropout(0.1),
            nn.Linear(10, 2)
            )

    def forward(self, x): # [bs, freq, frame] ex)[bs, 40, 300]
        
        x = x.permute(0, 2, 1) # [B, C, T] to [B, T, C] for pack_padded_sequence

        output, h_n = self.rnn(x)

        frame_output = self.frame_linear(output).permute(0, 2, 1)
        
        batch = x.size()[0]
        num_layers = self.rnn.num_layers
        hidden_size = self.rnn.hidden_size
        
        # hidden = h_n_2.view(num_layers, 2, batch, hidden_size)
        hidden = h_n.view(3, 2, batch, hidden_size)        
        
        last_hidden = hidden[-1] # forward + backward
        last_hidden = last_hidden.mean(dim=0)
        
        murmur_output = self.murmur_linear(last_hidden)
        outcome_output = self.murmur_linear(last_hidden)
        
        return frame_output, murmur_output, outcome_output # back to [B, C, T]

    @property
    def output_fs(self):
        return int(1 / WINDOW_STEP)
    
    
model = GRU_multi_output()



input = torch.randn(32, 40 ,300)

frame_output, murmur_output, outcome_output = model(input)

frame_output.shape, murmur_output.shape, outcome_output.shape


(torch.Size([32, 5, 300]), torch.Size([32, 2]), torch.Size([32, 2]))

## T4 전처리

In [25]:
class Custome_T4_dataset(torch.utils.data.Dataset):                           
    def __init__(self, data_folder, recording_paths, timings, murmur_labels, outcome_labels, sequence_length, cache=None):
        self.data_folder = pathlib.Path(data_folder)  
        self.recording_paths = recording_paths   
        self.timings = timings
        self.murmur_labels = murmur_labels
        self.outcome_labels = outcome_labels
        self.sequence_length = sequence_length
        self.cache = cache        
        assert len(self.recording_paths) == len(self.murmur_labels)

        
    def __len__(self):
        return len(self.recording_paths)

    def __getitem__(self, idx):
        
        filename = self.recording_paths[idx]  
        label = self.murmur_labels[idx] 
        timing = self.timings[idx] 
        outcome_label = self.outcome_labels[idx] 
        
    # if self.cache is None or filename not in self.cache:
        filepath = self.data_folder / filename
        fs, recording = spio.wavfile.read(filepath.with_suffix(".wav"))

        recording = torch.as_tensor(recording.copy())
        features, fs_features = calculate_features(recording, fs)

        segmentation_path = filepath.with_suffix(".tsv")
        segmentation_df = pd.read_csv(segmentation_path, sep="\t", header=None)
        segmentation_df.columns = ["start", "end", "state"]
        segmentation_df["start"] = (segmentation_df["start"] * fs_features).apply(lambda x: int(x)) # 수정
        segmentation_df["end"] = (segmentation_df["end"] * fs_features).apply(lambda x: int(np.ceil(x))) # 수정
        
        # features.shape[-1] : frame길이
        
        # 기존
        # segmentation_label = torch.zeros(features.shape[-1], dtype=torch.long)
        
        # 수정
        segmentation_label = torch.zeros(5, features.shape[-1]) # BCE_Loss 쓸 꺼니까 float처리
        
        for row in segmentation_df.itertuples():
            # IN:  noise =  0, S1 = 1, systole = 2, S2 = 3, diastole = 4
            # OUT: noise = -1, S1 = 0, systole = 1, S2 = 2, diastole = 3, murmur = 4
            if row.state == 0:
                # Noise
                segmentation_state = -1
            if row.state == 1:
                # S1
                segmentation_label[row.state -1 , row.start : row.end] = 1.0
            elif row.state == 3:
                # S2
                segmentation_label[row.state -1 , row.start : row.end] = 1.0
            elif row.state == 2:
                # Systole
                segmentation_label[row.state -1 , row.start : row.end] = 1.0
            elif row.state == 4:
                # Diastole
                segmentation_label[row.state -1 , row.start : row.end] = 1.0

            #segmentation_label[row.start : row.end] = segmentation_state
            # segmentation_label[row.state, row.start : row.end] = 1.0  # 4
            
            
            
            # TODO: handle diastolic murmurs
            if (row.state == 2) and (label == "Present"):
                if timing == 'Early-systolic':
                    portion = [0, 0.5]
                elif timing == 'Holosystolic':
                    portion = [0, 1]
                elif timing == 'Mid-systolic':
                    portion = [0.25, 0.75]
                elif timing == 'Late-systolic':
                    portion = [0.5, 1]
                else:
                    portion = [0, 0]
                    #print(f"Warn: Got timing {timing} for file {filename}")
                    # Diastolic
                    #raise ValueError(f"ERROR: Got timing {timing} for file {filename}")

                state_duration = row.end - row.start # TODO: inclusive?
                start = int(row.start + portion[0] * state_duration)
                end = int(np.ceil(row.start + portion[1] * state_duration))
                
                # murmur 라벨링
                #segmentation_label[row.state, start : end] = 4
                segmentation_label[4, start : end] = 1.0
                
        
        if label == "Absent":
            murmur_label = torch.zeros(1).long()
        else:
            murmur_label = torch.ones(1).long()
        
        
        if outcome_label == "Normal":
            outcome_label = torch.zeros(1).long()
        else:
            outcome_label = torch.ones(1).long()
            
        if self.sequence_length is not None:
            
            random_start = torch.randint(
                low=0, high=max(features.shape[-1] - self.sequence_length, 1), size=(1,)
            ).item()
            
            features = features[..., random_start : random_start + self.sequence_length]
            segmentation_label = segmentation_label[...,
                random_start : random_start + self.sequence_length
            ]

        return features, segmentation_label, murmur_label, outcome_label, filename

In [26]:
train_recording_df.patient_murmur_label.unique(), train_recording_df.outcome_label.unique()

(array(['Absent', 'Present'], dtype=object),
 array(['Abnormal', 'Normal'], dtype=object))

In [27]:
shared_feature_cache = {}

train_dataset = Custome_T4_dataset(
                data_folder= data_folder, 
                recording_paths= train_recording_df.index, 
                murmur_labels= train_recording_df.patient_murmur_label, 
                outcome_labels= train_recording_df.outcome_label,
                timings= train_recording_df.murmur_timing,
        sequence_length= None, # Default: int(TRAINING_SEQUENCE_LENGTH / WINDOW_STEP)
        cache=shared_feature_cache
    )


val_dataset = Custome_T4_dataset(
                data_folder= data_folder, 
                recording_paths= val_recording_df.index, 
                murmur_labels= val_recording_df.patient_murmur_label, 
                outcome_labels= val_recording_df.outcome_label,
                timings= val_recording_df.murmur_timing,
        sequence_length= None, # Default: int(TRAINING_SEQUENCE_LENGTH / WINDOW_STEP)
        cache=shared_feature_cache
    )


In [28]:
for idx, (features, segmentation_label, murmur_label, outcome_label, filename) in enumerate(train_dataset):
    print(features.shape, segmentation_label.shape, murmur_label.shape, outcome_label.shape, filename)

torch.Size([40, 1178]) torch.Size([5, 1178]) torch.Size([1]) torch.Size([1]) 2530_AV
torch.Size([40, 651]) torch.Size([5, 651]) torch.Size([1]) torch.Size([1]) 2530_PV
torch.Size([40, 748]) torch.Size([5, 748]) torch.Size([1]) torch.Size([1]) 2530_TV
torch.Size([40, 1332]) torch.Size([5, 1332]) torch.Size([1]) torch.Size([1]) 2530_MV
torch.Size([40, 906]) torch.Size([5, 906]) torch.Size([1]) torch.Size([1]) 9979_AV
torch.Size([40, 1011]) torch.Size([5, 1011]) torch.Size([1]) torch.Size([1]) 9979_PV
torch.Size([40, 1143]) torch.Size([5, 1143]) torch.Size([1]) torch.Size([1]) 9979_TV
torch.Size([40, 1056]) torch.Size([5, 1056]) torch.Size([1]) torch.Size([1]) 9979_MV
torch.Size([40, 512]) torch.Size([5, 512]) torch.Size([1]) torch.Size([1]) 13918_AV
torch.Size([40, 468]) torch.Size([5, 468]) torch.Size([1]) torch.Size([1]) 13918_PV
torch.Size([40, 328]) torch.Size([5, 328]) torch.Size([1]) torch.Size([1]) 13918_TV
torch.Size([40, 1437]) torch.Size([5, 1437]) torch.Size([1]) torch.Size([1

In [29]:
shared_feature_cache = {}

train_dataset = Custome_T4_dataset(
                data_folder= data_folder, 
                recording_paths= train_recording_df.index, 
                murmur_labels= train_recording_df.patient_murmur_label, 
                outcome_labels= train_recording_df.outcome_label,
                timings= train_recording_df.murmur_timing,
        sequence_length= None, # Default: int(TRAINING_SEQUENCE_LENGTH / WINDOW_STEP)
        cache=shared_feature_cache
    )

val_dataset = Custome_T4_dataset(
                data_folder= data_folder, 
                recording_paths= val_recording_df.index, 
                murmur_labels= val_recording_df.patient_murmur_label, 
                outcome_labels= val_recording_df.outcome_label,
                timings= val_recording_df.murmur_timing,
        sequence_length= None, # Default: int(TRAINING_SEQUENCE_LENGTH / WINDOW_STEP)
        cache=shared_feature_cache
    )


In [30]:
for idx, (features, segmentation_label, murmur_label, outcome_label, filename) in enumerate(train_dataset):
    #print(features.shape, segmentation_label.shape, murmur_label.shape, outcome_label.shape, filename)
    print(segmentation_label.sum())

tensor(547.)
tensor(384.)
tensor(757.)
tensor(1375.)
tensor(335.)
tensor(703.)
tensor(348.)
tensor(905.)
tensor(563.)
tensor(354.)
tensor(386.)
tensor(431.)
tensor(439.)
tensor(897.)
tensor(992.)
tensor(360.)
tensor(246.)
tensor(366.)
tensor(725.)
tensor(340.)
tensor(509.)
tensor(753.)
tensor(842.)
tensor(1171.)
tensor(383.)
tensor(521.)
tensor(794.)
tensor(470.)
tensor(1480.)
tensor(1401.)
tensor(1415.)
tensor(988.)
tensor(421.)
tensor(771.)
tensor(459.)
tensor(164.)
tensor(1273.)
tensor(1254.)
tensor(1203.)
tensor(824.)
tensor(500.)
tensor(657.)
tensor(365.)
tensor(446.)
tensor(476.)
tensor(688.)
tensor(858.)
tensor(669.)
tensor(308.)
tensor(268.)
tensor(262.)
tensor(508.)
tensor(895.)
tensor(765.)
tensor(524.)
tensor(1169.)
tensor(587.)
tensor(421.)
tensor(453.)
tensor(875.)
tensor(448.)
tensor(843.)
tensor(437.)
tensor(341.)
tensor(630.)
tensor(486.)
tensor(880.)
tensor(1501.)
tensor(1661.)
tensor(344.)
tensor(1600.)
tensor(993.)
tensor(201.)
tensor(1549.)
tensor(1124.)
tensor(1107

In [40]:
def collate_fn_slice_add(x):
    all_features = []
    all_frame_labels = []
    all_murmur_labels = []
    all_outcome_labels = []
    all_filenames = []
    
    frame_len = int(300)
    
    
    all_features = [] # [len, 40, 300]
    all_frame_labels = [] # [len, 5, 300]
    
    for idx, (feature, segmentation_label, murmur_label, outcome_label, filename) in enumerate(x): # train_dataset
        
        current_len = feature.size(1)
        
        # 더 짧을 때        
        if current_len < frame_len:
            
            diff = frame_len - current_len
            
            left_pad = random.randint(0, diff)
            right_pad = diff - left_pad
            
            padded_feature = F.pad(feature, (left_pad, right_pad), 'constant', 0.0)
            segmentation_label = F.pad(segmentation_label, (left_pad, right_pad), 'constant', -1)  # 이거 -1로 바꿔야겠네
            
            # 추가
            all_features.append(padded_feature)
            all_frame_labels.append(segmentation_label)
            all_murmur_labels.append(murmur_label)
            all_outcome_labels.append(outcome_label)
            all_filenames.append(filename)
        else:
            #diff = current_len - frame_len
        
            # start = random.randint(0, diff)
            # padded_feature = feature[:, start : start + 300]
            # segmentation_label = segmentation_label[:, start : start + 300]
            
            for j in range(current_len // 300): 
                sliced_feature = feature[:, j * 300 : (j+1) * 300]
                sliced_seg_label = segmentation_label[:, j * 300 : (j+1) * 300]

                
                print(sliced_feature.shape)
                print(sliced_seg_label.shape)
                
                
                # 추가
                all_features.append(sliced_feature)
                all_frame_labels.append(sliced_seg_label)
                all_murmur_labels.append(murmur_label)
                all_outcome_labels.append(outcome_label)
                all_filenames.append(filename)
            

    all_features = torch.stack(all_features).float() 
    all_frame_labels = torch.stack(all_frame_labels).float() # BCE_loss 
    all_murmur_labels = torch.tensor(all_murmur_labels).long() # CE_loss
    all_outcome_labels = torch.tensor(all_outcome_labels).long() # CE_loss
        
    return all_features, all_frame_labels, all_murmur_labels, all_outcome_labels, all_filenames

In [41]:
padded_features, padded_labels, all_murmur_labels, all_outcome_labels, all_filenames = collate_fn_slice_add(train_dataset)

torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size([40, 300])
torch.Size([5, 300])
torch.Size(

In [34]:
for feature, seg_label, murmur_label, outcome_label, filename in zip(padded_features, padded_labels, all_murmur_labels, all_outcome_labels, all_filenames):
    
    if tuple(feature.size()) != (40, 300) or tuple(seg_label.size()) != (5, 300):
        print(feature.shape, seg_label.shape)
    
    print(feature.shape, seg_label.shape)
    #print(murmur_label, outcome_label, filename)

torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size([40, 300]) torch.Size([5, 300])
torch.Size(

In [35]:
def collate_slice_add(x):
    
    all_features = []
    all_frame_labels = []
    all_murmur_labels = []
    all_outcome_labels = []
    all_filenames = []
    
    frame_len = int(300)
    
    all_features = torch.zeros(len(x), 40, 300)
    all_frame_labels = torch.zeros(len(x), 5, 300)
    
    for idx, (feature, segmentation_label, murmur_label, outcome_label, filename) in enumerate(x): # train_dataset
        
        current_len = feature.size(1)
        
        # 더 짧을 때        
        if current_len < frame_len:
            
            diff = frame_len - current_len
            
            left_pad = random.randint(0, diff)
            right_pad = diff - left_pad
            
            padded_feature = F.pad(feature, (left_pad, right_pad), 'constant', 0.0)
            segmentation_label = F.pad(segmentation_label, (left_pad, right_pad), 'constant', 0.0)
            
            
            
            
        else:
            for num in range(current_len // frame_len):
                
        
        
        
        all_features[idx] = padded_feature
        all_murmur_labels.append(murmur_label)
        all_outcome_labels.append(outcome_label)
        all_filenames.append(filename)
    
    
    return all_features, all_frame_labels, all_murmur_labels, all_outcome_labels, all_filenames

IndentationError: expected an indented block after 'for' statement on line 33 (2849106017.py, line 38)